<a href="https://colab.research.google.com/github/egynk3/simpleNeuralNetwork/blob/main/SimpleNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Creating a neural network to determine final height
# Import PyTorch and other relevant libraries
import torch
import torch.nn as nn
import torch.optim as optim
# Import all remaining packages
import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm
from scipy.io.wavfile import write

#get data
#normalize data
#define training and validation sets
